# Counting Rubik's Snake Shapes

*Dmytro Fedoriaka, August 2024*

## 1. Introduction

Rubik's Snake [1] is a puzzle toy consisting of wedges connected by joints. The original toy released in 1981 had 24 wedges, but it can have any number of wedges.

With some natural constraints imposed, the Rubik's Snake has a very large, but finite number of possible configurations (also called shapes).

The number of shapes for the 24-wedge puzzle was first computed in 2011 (see [2]), but the result was incorrect until it was revised in 2022.

The goal of this project is to **compute the number of n-wedge Rubik's Snake Shapes** for n up to 26, and also to confirm the result in [2].

## 2. Definitions and the approach

### 2.1. Problem statement

For general definitions of Rubik's Snake and its elements, see [1] or [2].

In this project we want to compute a number $S_n$ which is a number of distinct Rubik's Snake shapes. We need to precisely define what is a shape and what it means for two shapes to be distinct. We will give two equivalent definitions.

**Problem statement 1.** (using definitions from [2])
 * Two wedges are adjacent if they share a square side.There are 4 configurations of adjacent wedges: straight (0), right (1), left (3), opposing (2), each a multiple of 90-degree twists.
 * Two wedges are neighbors if they’re connected through a joint.
 * A (proper) shape is a Rubik's Snake configuration in which all wedges are adjacent to their neighbors. Informally, this means there is no bending, and no rotations by angles other than multiples of 90°.
 * One end of Rubik's snake is "head", the other is "tail" and they are distinguishable.
 * Two shapes are the same if and only if they can be matched by a translation and rotation, such that head is matched to head and tail is matched to tail. Otherwise they are distinct.
 * $S_n$ is the number of distinct proper shapes of a Rubik's snake with $n$ wedges.
 
However, there is a more convenient definition using a formula. A shape of $n$-wedge snake can be fully determined by a *formula*, which is a string of $n-1$ characters "0","1","2" or "3". It instructs you how to rotate all $n-1$ joints of the Snake (see section "Notation" on page 19 in [3]).

Let's fix a position of the first wedge, and add the next edges as instructed by formula. If at some point it would require adding a wedge such that it would have intersection with another wedge of *positive volume*, the formula is *invalid*. Otherwise, the formula is *valid*.

**Problem statement 2.** $S_n$ is the number of strings from $\{0,1,2,3 \}^{n-1}$ that are valid formulas.


### 2.2. Definitions

**Grid.** There is a grid of $(2K+1)^3$ unit cubes. Each cube has integer coordinates $x,y,z$, each in range $[0,2K]$. The cube $(K,K,K)$ is a center cube. For efficiency, we encode cube $(x,y,z)$ by a number 
$(dx \cdot x+dy \cdot y + dz \cdot z)$, where $dx=1$, $dy=2K+1$, $dy=(2K+1)^2$. We take $K=\lfloor \frac{N_{max}}{2} \rfloor$, where $N_{max}$ is maximal possible length of snake. It's guaranteed that when the first wedge (head) is placed in a center cube $(K,K,K)$, every possible shape will fit inside the grid.

**Cube geometry.** Each unit cube has 6 faces. They have face_ids defined as follows (face_ids of opposed faces add up to 5):
* The face facing -y direction has face_id=0.
* The face facing -z direction has face_id=1.
* The face facing -x direction has face_id=2.
* The face facing +x direction has face_id=3.
* The face facing +z direction has face_id=4.
* The face facing +y direction has face_id=5.

**Cube occupancy type.** Each cube in the grid can have 14 occupancy types, encoded by integers from 0 to 13:
* It can be empty. This is encoded as 0.
* It can be fully occupied. This means there are two "complementary" wedges in it (that is, intersection of the edges has zero volume). This is encoded as 13.
* It can be occupied by one edge. There are 12 possible wedge orientations, so these occupancy types are encoded by integers from 1 to 12. These numbers are assigned in such a way that if two occupancy types are complementary, their codes add up to 13.

**Wedge.** A wedge is an elementary part of Rubik's snake. It is a pentahedron of volume 0.5. It's faces are: 2 unit squares, 2 triangles and one $1 \times \sqrt{2}$ rectangle. In a (valid) shape, all wedges are placed such that every wedge is fully inside one cube and its two unit squares exactly match some two faces of this cube. Therefore, a wedge position is fully determined by coordinates of a cube and a pair of face_ids denoting which of the cube's faces coincide with the wedge's unit squares.

**Wedge ID.**
There are 24 possible ordered pairs of faces of a cube that share one edge. This is how much different wedge orientations there can be within one cube. These are mapped to "wedge IDs", which are integers 1..12, 17..28 such that the lowest 4 bits are equal to the occupancy type of this wedge. 

These conventions make it convenient to check whether wedge can be placed in certain cube:
* Wedge with wedge_id=x cab be placed in cube with occupancy type y if $y=0 \vee (x\%16)+y=13$.
* When placing a wedge with wedge_id=x into a cube, we must add (x%16) to the cube's occupancy type.
* When removing a wedge with wedge_id=x from a cube, we must subtract (x\%16) from the cube's occupancy type.

**Wedge rotations.** Given the coordinates and wedge_id of $i$-th wedge, coordinates of the $(i+1)$-th wedge are uniquely determined by geometry (it will be in one of 6 adjacent cubes, and which one is defined by the second face_id of previous wedge). There are 4 possible $wedge_ids$ depending on rotation angle (there are 4 possible rotation angles - 0°, 90°, 180°, 270°, encoded by integers 0,1,2,3, this is standard convention used in formulas, see p.21 in [3]). We precompute the mapping from (previous wedge_id, rotation) to next wedge_id.

### 2.3. Algorithm

The general approach is straightforward: recursively enumerate all shapes, building them by adding one wedge at a time, and terminating recursion branch when first spatial conflict happened. There already is open-source program that does that (see [4]). However, in this project I added some optimizations that makes it about 1000x faster that code in [4]. Most importantly, my model of encoding wedges allows to avoid any geometric computations during the search, which are all replaced by lookups in pre-computed arrays.

Let's describe a recursive algorithm for enumerating all shapes. We start with placing one wedge at the center cube. Then we recursively call the following procedure count_shapes_rec that is called for some snake prefix:
* Increment S[i], where $i$ is the current snake's length.
* If $i=N_{max}$, exit recursion.
* Consider all 4 rotations for the next wedge. Check which of them will result in a valid shape (that is, they don't conflict with occupancy type of the cube they will be placed into. For each valid rotation, place the next wedge, recursively call count_shapes_rec, then remove the placed edge.

This algorithm computes values of $S_n$ for all $n$ up to $N_{max}$.

There are also some non-asymptotic optimizations:
* If the next wedge is last, and the next cube is empty, just add 4 to S[i+1].
* If the next wedge is next-to-last, and the next cube is empty, and all cubes around it, except the current cube, are also empty, all 64 2-wedge tails are possible. So add 4 to S[i+1] and add 64 to S[i+2].
* The computation is parallelized over prefixes of the first 2 rotations.
* Some of these prefixes result in the same result due to reflection symmetry, so we can compute them only once and multiply the answer for them by 2 (which allows us to consider only 10 instead of 16 prefixes).
* Numba [5] is used to compile Python code, which significantly speeds it up. 

## 3. Code and computation

Below is the code for computing values of $S_n$. It is written in Python 3 using only 2 libraries (NumPy and Numba). 

This code was run on AMD Ryzen 7 5825U, and it took 1 hour to get results up to $n=24$, and 13.4 hours to get results up to $n=26$.

Results for $n \ge 27$ obtained by the same code on a different machine (took 6 days to compute value for for $n=28$).

In [1]:
import numpy as np
import time
import numba
import multiprocessing

# Prepare the grid.
MAX_N=26
K = 2*(MAX_N//2)
box_size = 2*K+1
dx,dy,dz=1,box_size,box_size**2
CENTER_COORD = K*(dx+dy+dz)

# Pre-calculate geometry.
CUBE = [[1,3,4,2],[0,2,5,3],[0,4,5,1],[0,1,5,4],[0,3,5,2],[1,2,4,3]]
DELTAS = np.array([dy,dz,dx,-dx,-dz,-dy]) # "+y","+z","+x","-x","-z","-y"
WEDGE_ID_TO_FACE_IDS = dict()
FACE_IDS_TO_WEDGE_ID = dict()

def register_wedge(f1, f2, wedge_id):
  WEDGE_ID_TO_FACE_IDS[wedge_id] = (f1, f2)
  FACE_IDS_TO_WEDGE_ID[(f1, f2)] = wedge_id

for i, (f1, f2) in enumerate([(0,1),(0,2),(0,3),(0,4),(1,2),(1,3)]):
  register_wedge(f1,f2, i+1)
  register_wedge(f2,f1, i+1+16)
  register_wedge(5-f1,5-f2, 13-(i+1))
  register_wedge(5-f2,5-f1, 13-(i+1)+16)
  
WEDGE_ID_TO_NEXT_DELTA = np.zeros(36, dtype=np.int64)
ROT_AND_WEDGE_ID_TO_NEXT_WEDGE_ID = np.zeros(36*4, dtype=np.int64)
for f1 in range(6):
  for f2 in CUBE[f1]:
    wedge_id = FACE_IDS_TO_WEDGE_ID[(f1,f2)]
    f1p=5-f2
    f2p=[5-f1,0,f1,0]
    f2p[1]=CUBE[f1][(CUBE[f1].index(f2)+1)%4]
    f2p[3]=5-f2p[1]
    WEDGE_ID_TO_NEXT_DELTA[wedge_id] = DELTAS[f1p]
    for rot in range(4):
      ROT_AND_WEDGE_ID_TO_NEXT_WEDGE_ID[wedge_id+rot*36] = FACE_IDS_TO_WEDGE_ID[(f1p, f2p[rot])]

@numba.jit("i8(i8,i8)", inline="always")
def encode_wedge(coord, wedge_id):
  return (coord<<6) + wedge_id
      
@numba.jit("(i8,i8,i8[:],i8[:])", inline="always")
def push_wedge(wedge_coord, wedge_id, wedges, cubes):
  wedges[0] -= 1
  wedges[wedges[0]] = encode_wedge(wedge_coord, wedge_id)
  cubes[wedge_coord] += wedge_id&15
  
@numba.jit("i8(i8,i8)", inline="always")
def get_next_wedge_coord(last_wedge_id, last_wedge_coord):
  return last_wedge_coord + WEDGE_ID_TO_NEXT_DELTA[last_wedge_id]

@numba.jit("i8(i8,i8)", inline="always")
def get_next_wedge_id(last_wedge_id, rot):
  return ROT_AND_WEDGE_ID_TO_NEXT_WEDGE_ID[last_wedge_id+36*rot]

@numba.jit("(i8[:],i8[:],i8[:])")
def count_shapes_rec(wedges, cubes, total_count):
  last_wedge_index = wedges[0]
  total_count[last_wedge_index] += 1
  if last_wedge_index == 1: return  # Full length shape, stop recusrion.
  last_wedge = wedges[wedges[0]]
  last_wedge_id = last_wedge&63
  last_wedge_coord = last_wedge>>6
  next_wedge_coord = get_next_wedge_coord(last_wedge_id,last_wedge_coord)
  next_cube_occupancy_type = cubes[next_wedge_coord]
  
  if next_cube_occupancy_type==0 and last_wedge_index == 2:
    total_count[1] += 4
    return
  if next_cube_occupancy_type==0 and last_wedge_index == 3:
    c=next_wedge_coord
    s = cubes[c-dx]+cubes[c+dx]+cubes[c-dy]+cubes[c+dy]+cubes[c-dz]+cubes[c+dz]
    if s==cubes[last_wedge_coord]:
      total_count[2] += 4
      total_count[1] += 16
      return  
  
  for rot in range(4):
    next_wedge_id = get_next_wedge_id(last_wedge_id, rot)
    next_wedge_occupancy_type = next_wedge_id&15
    can_push = next_cube_occupancy_type==0 or (next_cube_occupancy_type+next_wedge_occupancy_type == 13)
    if can_push:
      push_wedge(next_wedge_coord, next_wedge_id, wedges, cubes)
      count_shapes_rec(wedges, cubes, total_count)
      cubes[next_wedge_coord] -= next_wedge_occupancy_type  # pop
      wedges[0] += 1                                        # pop

def count_shapes_with_prefix(n, prefix=[]):
  total_count = np.zeros(n+1, dtype=np.int64)
  wedges=np.zeros(n+1, dtype=np.int64)
  wedges[0]=n+1 # wedges[0] indicates last wedge index
  cubes = np.zeros(box_size**3, dtype=np.int64)
  push_wedge(CENTER_COORD, FACE_IDS_TO_WEDGE_ID[(0,3)], wedges, cubes)  # Initial wedge.
  assert len(prefix) <= 2
  for rot in prefix:
    last_wedge = wedges[wedges[0]]
    last_wedge_coord,last_wedge_id = last_wedge>>6,last_wedge&63 
    next_wedge_coord = get_next_wedge_coord(last_wedge_id,last_wedge_coord)
    next_wedge_id = get_next_wedge_id(last_wedge_id, rot)
    push_wedge(next_wedge_coord, next_wedge_id, wedges, cubes)
  count_shapes_rec(wedges, cubes, total_count)
  return total_count[1:][::-1]

def f(x):
  n,prefix,mul = x
  return mul*count_shapes_with_prefix(n,prefix=prefix)

def count_shapes(n):
  assert 3<=n<=MAX_N
  tasks = [([0,0],1),([0,1],2),([0,2],1),([2,0],1),([2,1],2),([2,2],1)]+[([1,i],2) for i in range(4)]
  tasks = [(n,prefix,mul) for prefix, mul in tasks]
  with multiprocessing.Pool() as p:
    ans = np.sum(p.map(f, tasks), axis=0)
  ans[0],ans[1]=1,4
  return ans

# Quick test for n=12.
t0=time.time()
assert np.all(count_shapes(12) == [1,4,16,64,241,920,3384,12585,46471,172226,633138,2333757])
print("OK, %fs" % (time.time()-t0))

OK, 0.073505s


In [2]:
for n in range(18,27):
  t0=time.time()
  counts = count_shapes(n)
  print("n=%d, time=%fs" % (n, time.time()-t0))
  print(counts)

n=18, time=1.489173s
[         1          4         16         64        241        920
       3384      12585      46471     172226     633138    2333757
    8561679   31462176  115247629  422677188 1546186675 5661378449]
n=19, time=5.373640s
[          1           4          16          64         241         920
        3384       12585       46471      172226      633138     2333757
     8561679    31462176   115247629   422677188  1546186675  5661378449
 20689242550]
n=20, time=20.767645s
[          1           4          16          64         241         920
        3384       12585       46471      172226      633138     2333757
     8561679    31462176   115247629   422677188  1546186675  5661378449
 20689242550 75663420126]
n=21, time=76.771875s
[           1            4           16           64          241
          920         3384        12585        46471       172226
       633138      2333757      8561679     31462176    115247629
    422677188   1546186675   5661378

## 4. The main result

The first 26 terms of the sequence $S_n$ are: 
  **1, 4, 16, 64, 241, 920, 3384, 12585, 46471,
  172226, 633138, 2333757, 8561679, 31462176, 115247629,
 422677188, 1546186675, 5661378449, 20689242550, 75663420126, 276279455583,
 1009416896015, 3683274847187, 13446591920995, 49037278586475, 178904588083788,
 652111697384508, 2377810831870022**.
 
I verified this result using the following sources:
* My value for $S_{24}$ matches the value published in [2].
* I computed the first 14 values using [4], and they match my values.

## 5. Discussion

### 5.1. Computing further terms

The exhaustive search is easily parallelizable. One needs to enumerate prefixes of some length $k$ (that are also valid shapes), independently compute shapes starting with each prefix and add the results. Using this method, higher values can be computed using CUDA and/or distributed computing.

However, this will only give us a few more terms, because no matter what is computational setup, computing the next term will take 4 times longer than the previous. Can this number be computed asymptotically faster than $O(4^n)$? I don't know (but I tend to think the answer is negative). Below are some ideas that I considered.

**Dynamic programming.**

Let's define a profile of a shape as a pair of two things:
  * coordinate and ID of the last wedge,
  * occupancy types for all cubes.
  
Then we can group all shapes by their profiles and only keep track of a number of shapes for the same profile. This information for profiles for $n$ will be sufficient to compute profiles and shape counts for $n+1$, so we can use profile dynamic programming. However, when I tried to implement this, I found that there are still too many distinct profiles for given $n$ - almost as many as shapes, so this won't be faster.

**Meet-in-the-middle.**

Can we enumerate all shapes for length $n$ and then efficiently combine them to get the number of shapes for length $2n$, so we can get asymptotics of $O(2^n)$ instead of $O(4^n)$?


### 5.2. Asymptotic behavior of $S_n$

It can be easily shown that $2^{n-1} \le S_n \le 4^{n-1}$. For more detailed analysis and tighter bounds, see this [notebook](asymptotic.ipynb).


### 5.3. Other related sequences

There are a lot of other sequences that can be defined for Rubik's Snake.

#### 5.3.1. Symmetries

Our definition deliberately doesn't "deduplicate" any shapes: two shapes of length $n$ having different formulas (strings of lengths $n-1$) are considered different. However, we can account for some symmetries.

**Reversal**. There are 3 alternative definitions for this:
  * If two shapes have the formulas that are reverses of each other, consider them the same. 
  * If two shapes can be matched by rotation and translation, consider them the same.
  * Count different shapes, without distinguishing "head" and "tail" of the snake.

**Reflection** (also called "chirality" in [4]). If one shape is a mirror reflection of another, consider them the same shape.

Here are some sequences we can get (I computed the first 12 terms using code from [4]):

* Distinct shapes up to reversal: $1,4,10,40,127,490,1718,6403,23328,86514,316919,1168357, \dots$. See [this notebook](count-shapes-with-reversal.ipynb) for more terms and detailed computation with code.
* Distinct shapes up to reflection: $1,3,10,36,127,472,1714,6333,23305,86238,316794,1167283, \dots$.
* Distinct shapes up to reversal and reflection: $1,3,7, 24,70, 258,880, 3247,11737,43411,158703,584716, \dots$.

Instead of counting shapes "up to" some symmetry we can count shapes *having* certain symmetry (i.e. shapes that are fixed points for corresponding transformation), but these two are related by a formula $2 D_n = S_n + F_n$, where $D_n$ is number of distinct shapes up to a symmetry, and $F_n$ is number of shapes fixed by a symmetry.

#### 5.3.2. Loops

A shape is a loop (for $n>=4$) if the first and last wedge are adjacent to each other.

The following sequences are possible:
* Number of valid formulas of length $n-1$, describing loops.
* Number of loops up to cyclic shifts. This means we imagine that head and tail are joined (with a mechanical joint) just like any other pair of adjacent wedges, and we forget where exactly this joint is.

[3] states that the number of loops for $n=24$, up to cyclic shifts and reversals, is 63'970'851.

There are much fewer loops than shapes, and we can enumerate them faster than $O(4^n)$. In particular, we can use the meet-in-the-middle approach.

If we ignore the position of head and tail and look at the shape as a set of wedges, some loops can have rotational symmetries along x/y/z axes by 90° and 180° degrees, and/or central symmetry. We can count all shapes having some of these symmetries.


#### 5.3.3. Spatial constraints

[3] Gives some interesting spatial constraints on shapes, for example:
* Snake fits in a fixed bounding box (e.g. 3x3x3).
* Snake visits a specified number of cubes.

We can define a sequence for the number of shapes/loops for Rubik's snake with $n$ wedges, up to some symmetries, satisfying some constraints.


### 5.4. Physical realizability

Is it true that all enumerated shapes are possible to realize with a physical Rubik's Snake puzzle, without breaking it? This question is not well-defined mathematically. The answer might depend on a particular physical snake (and how loose the joints are). However, it would be interesting to see a formula that describes in a theoretically possible shape (that is, all rotations are multiple of 90° and no two wedges have intersection with positive volume), that would be impossible to construct in practice. 

Intuitively it seems that for $n=24$ the answer is yes, because the 24-wedge snake is not that long. What about higher values of $n$?


## References
1. [Rubik's Snake - Wikipedia](https://en.wikipedia.org/wiki/Rubik%27s_Snake)

2. Peter Aylett - Rubik's Snake Combinations - https://blog.ylett.com/2011/09/rubiks-snake-combinations.html

3. Soul of the Snake - https://drive.google.com/file/d/1T0LeSeMmBMcdJFBEUBhYD21rZQ6EBVkC/view

4. https://github.com/scholtes/snek

5. https://numba.pydata.org/